In [8]:
import pickle
import numpy as np
import pandas as pd
import math
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import seaborn as sns
import sys,os

In [9]:
sys.path.append(os.path.abspath(os.path.join('../utils')))
from cleaner import *
from helper import *
from plot import *
from outlier_handler import OutlierHandler

In [10]:
#read data
user_engagement_df = pd.read_csv("../data/user_engagement_analysis.csv")
user_engagement_df.head()

,MSISDN/Number,cluster,xdr Sessions,Dur. (ms),Total Data Volume (Bytes)
0,33601001722,2,1.0,116720.0,8.786906e+08
1,33601001754,1,1.0,181230.0,1.568596e+08
2,33601002511,1,1.0,134969.0,5.959665e+08
3,33601007832,1,1.0,49878.0,4.223207e+08
4,33601008617,2,2.0,37104.0,1.457411e+09


In [11]:
user_experience_df = pd.read_csv("../data/experience_analytics_data.csv")
user_experience_df.head()

,MSISDN/Number,cluster,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes)
0,33601001722,2,46.0,76.0,2895381.0
1,33601001754,1,31.0,99.0,7685875.5
2,33601002511,2,59.0,97.0,4150403.0
3,33601007832,2,84.0,248.0,2396.0
4,33601008617,1,119.0,43204.5,8047304.5


Engagement Score

In [12]:
eng_df = user_engagement_df.set_index('MSISDN/Number')[
    ['xdr Sessions', 'Dur. (ms)', 'Total Data Volume (Bytes)']]

In [13]:
def normalizer(df, columns):
    norm = Normalizer()
    return pd.DataFrame(norm.fit_transform(df), columns=columns)
def scaler(df, columns):
    minmax_scaler = MinMaxScaler()
    return pd.DataFrame(minmax_scaler.fit_transform(df), columns=columns)

In [14]:
normalized_metrics = scaler(eng_df, ['Total Avg RTT (ms)',
    'Total Avg Bearer TP (kbps)',
    'Total TCP Retrans. Vol (Bytes)'])
normalized_metrics.describe()

,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes)
count,105716.000000,105716.000000,105716.000000
mean,0.142654,0.386189,0.400389
std,0.260005,0.268673,0.251240
min,0.000000,0.000000,0.000000
25%,0.000000,0.207700,0.206669
50%,0.000000,0.303929,0.371524
75%,0.400000,0.524620,0.524002
max,1.000000,1.000000,1.000000


In [15]:
with open("../models/user_engagement_analysis.pkl", "rb") as f:
    kmeans1 = pickle.load(f)